# Application of Linear Programming Model

# Allocation Model

One of the simplest forms of linear programs that occurs widely in application might be termed **allocation models**. The main issue is how to divide or allocate a valuable resource among competing needs. The resource may be land, capital, time, fuel, or anything else of limited availability.

### Example: Forest service Allocation

The U.S. Forest Service has used just such an allocation model to address the sensitive task of managing 191 million acres of national forestland. The Forest Service must trade off timber, grazing, recreational, environmental, national preservation, and other demands on forestland. 

Models of a forest begin by dividing land into homogeneous *analysis areas*. Several *prescriptions* or land management policies are then proposed and evaluated for each. The optimization seeks the best possible allocation of land in the analysis areas to particular prescriptions, subject to forest-wide restrictions on land use.

![](forestallocation.png)

Table above provides details of the fictional, 788 thousand acre Wagonho National Forest that we model. Wagonho is assumed to have 7 analysis areas, each subject to 3 different prescriptions. The first prescription encourages timbering, the second emphasizes grazing, and the third preserves the land as wilderness. Using index dimensions, let

$$i \text{ be analysis area number } (i=1,...,7)$$
$$j \text{ be  prescription number } (i=1,...,3)$$

Table above provides values for all the following symbolic parameters:

$$s_i \text{: size of analysis area i (in thousands of acres) }$$
$$p_{ij} \text{: net present value (NPV) per acre of all uses in area i if managed under prescription j }$$
$$t_{ij} \text{: projected timber yield (in board feet per acre) of analysis area i if managed under prescription j }$$
$$g_{ij} \text{: projected grazing capability (in animal unit months per acre) of analysis area i if managed under prescription j }$$
$$w_{ij} \text{: wilderness index rating (0 to 100) of analysis area i if managed under prescription j }$$

We wish to find an allocation that maximizes net present value while producing 40
million board feet of timber, 5 thousand animal unit months of grazing, and keeping average wilderness index to at least 70.

**Decision Variables**

As in all such models, our Forest Service example seeks an optimal allocation of a valuable resource. Corresponding decision variables define the allocation.

*Principal decision variables in allocation models specify how much of the critical resource is allocated to each use*

Thus, our decision variable:

$$\text{number of thousands of acres in analysis area i managed by prescription j: } x_{ij}$$

**Objective function**

The Forest Service’s objective is to maximize total net present value (NPV). In terms of the defined notation, this is

$$\max \sum^7_{i=1}\sum^3_{j=1}p_{i,j}x_{i,j}$$

**Constraint**

Allocation: we must must assure that all acres of each analysis area are allocated
$$\sum^3_{j=1}x_{i,j}=s_i$$

Timber:

$$\sum^7_{i=1}\sum^3_{j=1}t_{i,j}x_{i,j}\geq 40000$$

Grazing:

$$\sum^7_{i=1}\sum^3_{j=1}g_{i,j}x_{i,j}\geq 5$$

Wilderness:

$$\frac{1}{788}\sum^7_{i=1}\sum^3_{j=1}w_{i,j}x_{i,j}\geq 70$$

### Julia Code

In [46]:
using JuMP, GLPK, LinearAlgebra

In [47]:
s=[75;90;140;60;212;98;113]
p=[503 140 203; 675 100 45; 630 105 40; 330 40 295; 105 460 120; 490 55 180; 705 60 400]
t=[310 50 0; 198 46 0; 210 57 0; 112 30 0; 40 32 0; 105 25 0; 213 40 0]
g=[0.01 0.04 0; 0.03 0.06 0; 0.04 0.07 0; 0.01 0.02 0; 0.05 0.08 0; 0.02 0.03 0; 0.02 0.04 0]
w=[40 80 95; 55 60 65; 45 55 60; 30 35 90; 60 60 70; 35 50 75; 40 45 95]

m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, x[1:7,1:3] >= 0) 
for i=1:7
    @constraint(m, sum(x[i,:]) == s[i]) # Allocation
end
@constraint(m, sum(t.*x) >=40000)
@constraint(m, sum(g.*x) >=5)
@constraint(m, sum(w.*x) >=70*788)
@objective(m, Max, sum(p.*x))
m

A JuMP Model
Maximization problem with:
Variables: 21
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 21 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 7 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [49]:
optimize!(m)
for i=1:7
    for j=1:3
        println("x",i,j,"=",round(value(x[i,j])))
    end
end
println("objective = ", objective_value(m))

x11=0.0
x12=0.0
x13=75.0
x21=90.0
x22=0.0
x23=0.0
x31=140.0
x32=0.0
x33=0.0
x41=0.0
x42=0.0
x43=60.0
x51=0.0
x52=154.0
x53=58.0
x61=0.0
x62=0.0
x63=98.0
x71=0.0
x72=0.0
x73=113.0
objective = 322515.0


## Operations Planning Models 

Another classic linear program form deals with operations planning. In organizations ranging from volunteer, to government, to manufacturing, to distribution, planners must decide what to do and when and where to do it. 

### Example: Canadian Forest Products Limited (CFPL) operations planning

Operations planning models become more complex when there are several stages of production. Activity at each stage consumes output of the preceding stage and creates input to the next stage.

![](cfpl1.png)

Canadian Forest Products Limited (CFPL) employed such a model to plan their production of plywood. Above figure shows the sequence of stages. Production begins by purchasing logs and peeling them into strips of thin “green” veneer. Green veneer can also be purchased directly. All green veneer is next dried, classified by quality, and in some cases **improved by patching knots and gluing thin strips together**. After the veneer has been cut into sheet sizes, several layers are glued and pressed to produce plywood. A final production step sands completed plywood and trims it to exact size for sale.

The objective of CFPL’s operations research analysis was to determine how to operate production facilities to maximize *contributed margin*: sales income less wood costs. Labor, maintenance, and other plant costs were assumed fixed. The principal constraint, other than limits on availability of wood and the market for various products, was the limited plant capacity to press plywood.

To have some numbers to work with, assume that logs are available from two vendors in “good” and “fair” qualities at the rate and price shown below. The table also shows the estimated yield in $\frac{1}{16}$- and $\frac{1}{8}$-inch green veneer of grades A, B, and C from peeling a log of the quality indicated.

![](cfpl2.png)

We can also purchase green veneer. Suppose that availabilities and purchase prices are as shown in the following table.

![](cfpl3.png)

Our version of CFPL will make just 6 products—all 4- by 8-foot sheets of plywood for the U.S. market. A final table shows the composition or **Bill of Materials** of each product in veneer sheets, the available market per month, and the time required to glue and press each sheet of plywood out of a monthly capacity of 4500 hours.

![](cfpl4.png)

**Decision Variable**

We begin a model for the CFPL case by choosing variables deciding how much of what to do. Index dimensions include:
$$q \text{: log quality = G for good, F for fair}$$
$$v \text{: log vendornumber = (1, 2)}$$
$$t \text{: veneer thickness = (1/16, 1/8, 1/4, 1/2)}$$
$$g \text{: veneer thickness = (A,B,C)}$$

To formulate the problem as a linear program, we will use four classes of (continuous) decision variables over these index dimensions:

$$w_{q,v,t} \text{: number of logs of quality q bought from vendor v and peeled into green veneer of thickness t per month}$$
$$x_{t,g} \text{: number of square feet of thickness t, grade g green veneer purchased directly per month}$$
$$y_{t,g,g'} \text{: number of sheets of thickness t veneer used as grade g' after drying and
processing from grade g green veneer per month}$$
$$z_{t,g,g'} \text{: number of sheets of thickness t, front veneer grade g, back veneer grade g' plywood pressed and sold per month}$$

### Continuous Variables for Integer Quantities

Readers who are studying LP modeling for the first time may be perplexed about the fact that the CFPL decision variables are all treated as continuous. Don’t quantities such as the number of logs and the number of sheets of plywood need to be
integers? Indeed, how can CFPL’s problem even be modeled as a linear program (which must have only continuous variable)?

Modeling physically integer quantities with continuous decision variables in this fashion is standard when optimal variable magnitudes are likely to be relatively large. If the LP-optimal number of plywood sheets sold of some type turns out to be, say, 953.2, there is little practical difficulty in rounding off to 953 sheets. After all, the costs, capacities, and other constants in the model are only estimates that contain a certain amount of error.

But we know that there is a big gain in tractability. Continuous optimization is almost always more efficient than discrete. To realize that gain without having much impact on the usability of optimal results, we choose to neglect integrality requirements.

Notice that this concession to tractability would be much more serious when decision variables were limited to, say, 0 and 1. If, for example, 0 means “do not build a facility” and 1 means “build it,” rounding continuous LP solutions could be much more problematic.

**Objective Function**

CFPL’s maximum contributed margin objective is easily expressed in terms of the decision variables above. We compute

$$\max - \text{(log costs) - (purchased veneer costs) + (sales income)}$$

**CFPL Constraints**

Log availability limits impose:

$$w_{G,1,1/16} + w_{G,1,1_8} \leq 200\\
w_{F,1,1/16} + w_{F,1,1/8} \leq 300\\
w_{G,2,1/16} + w_{G,2,1_8} \leq 100\\
w_{F,2,1/16} + w_{F,2,1/8} \leq 1000$$

purchased veneer availabilities imply that:

$$x_{1/16,A} \leq 5000\\
x_{1/16,B} \leq 25000\\
x_{1/16,C} \leq 40000\\
x_{1/8,A} \leq 10000\\
x_{1/8,B} \leq 40000\\ 
x_{1/8,C} \leq 50000$$

market sizes constraint:

$$z_{1/4,A,B} \leq 1000\\
z_{1/4,A,C} \leq 4000\\
z_{1/4,B,C} \leq 8000\\
z_{1/2,A,B} \leq 1000\\
z_{1/2,A,C} \leq 5000\\
z_{1/2,B,C} \leq 8000 $$

pressing capacity limit:

$$0.25(z_{1/4,A,B} + z_{1/4,A,C} + z_{1/4,B,C}) + 0.40(z_{1/2,A,B} + z_{1/2,A,C} + z_{1/2,B,C}) \leq 4500 $$

So far we have done nothing to link log and veneer purchasing at the beginning of the process to sales at the end. In fact, we have not used the processing variables $y_{t,g,g'}$ at all.
What makes operations planning models with several processing stages special is the need to provide such links through **balance constraints**.

> A **balance constraint** assures that in-flows equal or exceed out-flows for materials and products created by one stage of production and consumed by others.

The first family of balance constraints needed in the CFPL model involves green veneer. Assume that with trim losses, 35 square feet of green veneer is required for each 4-by 8-foot sheet of finished veneer. We then have for each thickness and grade

$$\text{(veneer from peeled logs) + (veneer purchased)} \geq 35\text{(sheets of veneer finished)}$$

Assuming that careful piecing and patching can permit green veneer of one grade to be used as the next higher, and veneer of any grade can be substituted for the next lower, we obtain the following six balance constraints for various grades and thicknesses of green veneer:

$$400w_{G,1,1/16} + 200w_{F,1,1/16} + 400w_{G,2,1/16} + 200w_{F,2,1/16} + x_{1/16,A} \geq 35y_{1/16,A,A} + 35y_{1/16,A,B}\\
700w_{G,1,1/16} + 500w_{F,1,1/16}+ 700w_{G,2,1/16} + 500w_{F,2,1/16} + x{1/16,B} \geq 35y_{1/16,B,A} + 35y_{1/16,B,B} + 35y_{1/16,B,C}\\
900w_{G,1,1/16} + 1300w_{F,1,1/16} + 900w_{G,2,1/16} + 1300w_{F,2,1/16}+ x_{1/16,C} \geq 35y_{1/16,C,B} + 35y_{1/16,C,C}\\
200w_{G,1,1/8} + 100w_{F,1,1/8} + 200w_{G,2,1/8} + 100w_{F,2,1/8} + x_{1/8,A} \geq 35y_{1/8,A,A} + 35y_{1/8,A,B}\\
350w_{G,1,1/8} + 250w_{F,1,1/8} + 350w_{G,2,1/8} + 250w_{F,2,1/8}+ x_{1/8,B} \geq 35y_{1/8,B,A} + 35y_{1/8,B,B} + 35y_{1/8,B,C}\\
450w_{G,1,1/8} + 650w_{F,1,1/8} + 450w_{G,2,1/8} + 650w_{F,2,1/8} + x_{1/8,C} \geq 35y_{1/8,C,B} + 35y_{1/8,C,C}$$

Six quite similar constraints enforce balance in sheets of finished veneer passing from the drying process to pressing:
$$\text{(sheets finished for use at this grade = sheets consumed in pressing}$$
We can make the constraints equalities this time because no veneer would ever
be finished unless it were going to be pressed. Again detailing for two thicknesses and three grades (other than the never-used 1/8-inch, grade A finished veneer) gives

$$y_{1/16,A,A} + y_{1/16,B,A} = z_{1/4,A,B} + z_{1/4,A,C} + z_{1/2,A,B} + z_{1/2,A,C}\\
y_{1/16,A,B} + y_{1/16,B,B} + y_{1/16,C,B} = z_{1/4,A,B} + z_{1/4,B,C} + z_{1/2,A,B} + z_{1/2,B,C}\\
y_{1/16,B,C} + y_{1/16,C,C} = z_{1/4,A,C} + z_{1/4,B,C} + z_{1/2,A,C} + z_{1/2,B,C}\\
y_{1/8,A,B} + y_{1/8,B,B} + y_{1/8,C,B} = z_{1/2,A,B} + z_{1/2,A,C} + z_{1/2,B,C}\\
y_{1/8,B,C} + y_{1/8,C,C} = z_{1/4,A,B} + z_{1/4,A,C} + z_{1/4,B,C} + 2z_{1/2,A,B} + 2z_{1/2,A,C} + 2z_{1/2,B,C}$$

## Homework

Try to do CFPL model:

![](cfpl5.png)

In [2]:
using GLPK, JuMP, LinearAlgebra
m = Model(with_optimizer(GLPK.Optimizer))
purchased = [5000 25000 40000; 10000 40000 50000]
market = [1000 4000 8000; 1000 5000 8000]
sqftPerLogW = zeros(2,2,2,3)
sqftPerLogW[:,:,:,1] = cat([400 400;200 200], [200 200;100 100], dims=3)
sqftPerLogW[:,:,:,2] = cat([700 700;500 500], [350 350;250 250], dims=3)
sqftPerLogW[:,:,:,3] = cat([900 900;1300 1300], [450 450;650 650], dims=3)
costPerLog = [340 190; 490 140]
costPerSqft = [1 0.3 0.1; 2.2 0.6 0.2]
price = [45 40 33; 75 65 50]                    # per sheet(1/4, 1/4, Ab, ac, bc)
@variable(m, w[1:2, 1:2, 1:2] >= 0)             # w(q,v,t)(log)
@variable(m, x[1:2, 1:3] >= 0)                  # x(t,g)(sqft)
@variable(m, y[1:2, 1:3, 1:3] >= 0)             # y(t,g,g)(sheets)
@variable(m, z[1:2, 1:3] >= 0)                  # z(t,g,g)(sheets)
@constraint(m, sum(w[1,1,:]) <= 200)    # log avalability
@constraint(m, sum(w[1,2,:]) <= 100)   
@constraint(m, sum(w[2,1,:]) <= 300)
@constraint(m, sum(w[2,2,:]) <= 1000)
for i=1:2
    for j=1:3
        @constraint(m, x[i,j] <= purchased[i,j])# purchased constraint
        @constraint(m, z[i,j] <= market[i,j])   # market constraint
    end
end
@constraint(m, 0.25*sum(z[1,:]) + 0.4*sum(z[2,:]) <= 4500)     #pressing constraint

@constraint(m, sum(sqftPerLogW[:,:,1,1].*w[:,:,1]) + x[1,1] >= 35*sum(y[1,1,1:2]))  # balance constraint
@constraint(m, sum(sqftPerLogW[:,:,1,2].*w[:,:,1]) + x[1,2] >= 35*sum(y[1,2,1:3]))
@constraint(m, sum(sqftPerLogW[:,:,1,3].*w[:,:,1]) + x[1,3] >= 35*sum(y[1,3,2:3]))

@constraint(m, sum(sqftPerLogW[:,:,2,1].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,1,1:2]))
@constraint(m, sum(sqftPerLogW[:,:,2,2].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,2,1:3]))
@constraint(m, sum(sqftPerLogW[:,:,2,3].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,3,2:3]))

@constraint(m, sum(y[1,1:2,1]) == sum(z[:,1:2]))
@constraint(m, sum(y[1,:,2]) == sum(z[:,1]) + sum(z[:,3]))
@constraint(m, sum(y[1,2:3,3]) == sum(z[:,2:3]))

# @constraint(m, sum(y[2,1:2,1]) == sum(z[:,1:2]))
@constraint(m, sum(y[2,:,2]) == sum(z[2,:]))
@constraint(m, sum(y[2,2:3,3]) == sum(z[1,:]) + 2*sum(z[2,:]))

@objective(m, Max, -(sum(costPerLog.*w[:,:,1]) + sum(costPerLog.*w[:,:,2])) - (sum(costPerSqft.*x)) 
           + sum(price.*z))

m

A JuMP Model
Maximization problem with:
Variables: 38
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 38 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 17 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y, z

In [4]:
optimize!(m)
println((objective_value(m)))

438357.1428571427
